In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import matplotlib
from tiatoolbox.wsicore.wsireader import (
    WSIReader,
    TIFFWSIReader,
    VirtualWSIReader,
)
from tiatoolbox.tools.patchextraction import get_patch_extractor
import cv2

|2024-06-13|11:41:38.932| [WARNING] /home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()

|2024-06-13|11:41:38.933| [WARNING] /home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details

In [ ]:
def erode_cell_mask(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    # closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    # opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.erode(mask, kernel, iterations=1)
    return mask

In [ ]:
img_dir = "/media/u1910100/Extreme SSD/data/tiger/cell_detection/dilation/patches/128/fold_1"
# img_dir = "/media/u1910100/Extreme SSD/data/tiger/cell_detection/dilation/tiles/fold_1"

img_names = os.listdir(img_dir)
for i in range(1000, 1500):
    img_name = img_names[i]

    fn = os.path.splitext(img_name)[0]
    if fn[-1] == "n":
        continue

    img_path = os.path.join(img_dir, img_name)

    img = np.load(img_path)

    # if img.shape[0] < 500:
    #     continue

    mask = img[:, :, 3]
    mask = erode_cell_mask(mask)
    # mask[mask==3] = 0
    # mask[mask==4] = 0
    # mask[mask==5] = 0
    # mask[mask==7] = 0

    # mask[mask==6] = 2

    # rgb_mask = np.zeros(shape=(mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    # rgb_mask[mask==1] = [255,0,0]
    # rgb_mask[mask==2] = [0,255,0]

    print(np.unique(mask))
    fig, axes = plt.subplots(1, 2, figsize=(5, 5))
    axes[0].imshow(img[:, :, 0:3])
    axes[1].imshow(mask)
    print(img_name)
    print(img.shape)
    print(np.count_nonzero(mask))
    plt.show()

In [ ]:
img_dir = "/media/u1910100/Extreme SSD/data/tiger/cell_detection/images"
mask_dir = "/media/u1910100/Extreme SSD/data/tiger/cell_detection/masks"

img_names = os.listdir(mask_dir)[:50]
img_names.sort()

for img_name in img_names:
    print(img_name)
    fig, axes = plt.subplots(1, 2, figsize=(10, 10))
    mask_path = os.path.join(mask_dir, img_name)
    img_path = os.path.join(img_dir, img_name)

    img = Image.open(img_path)
    print(img.size)
    axes[0].imshow(img)
    mask = Image.open(mask_path)
    mask = np.array(mask)
    print(np.unique(mask))
    axes[1].imshow(mask)
    # print(np.unique(mask))

    # mask[mask==3] = 0
    # mask[mask==4] = 0
    # mask[mask==5] = 0
    # mask[mask==7] = 0

    # mask[mask==6] = 2
    # print(np.unique(mask))
    # rgb_mask = np.zeros(shape=(mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    # rgb_mask[mask==1] = [255,0,0]
    # rgb_mask[mask==2] = [0,255,0]

    # axes[2].imshow(rgb_mask)
    plt.show()

In [ ]:
def crop_image(img, mask):
    crop_img = np.zeros_like(img)
    crop_mask = np.zeros_like(mask)

    binary_mask = np.zeros_like(mask)
    binary_mask[mask > 0] = 1

    contours, _ = cv2.findContours(
        binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    c = max(contours, key=cv2.contourArea)

    cv2.drawContours(crop_img, [c], 0, (1, 1, 1), -1)
    cv2.drawContours(crop_mask, [c], 0, 1, -1)

    img = img * crop_img
    mask = mask * crop_mask

    return img, mask


def merge_mask_classes(mask):
    merged_mask = np.zeros_like(mask)
    merged_mask[mask == 3] = 0
    merged_mask[mask == 4] = 0
    merged_mask[mask == 5] = 0
    merged_mask[mask == 7] = 0
    merged_mask[mask == 6] = 2
    return merged_mask

In [ ]:
img_dir = (
    "/home/u1910100/Documents/Tiger_Data/tissue_segmentation/tiles/fold_5"
)

for img_name in os.listdir(img_dir):
    # fig, axes = plt.subplots(1,3, figsize=(12,12))
    print(img_name)
    img_path = os.path.join(img_dir, img_name)

    file = np.load(img_path)
    mask = file[:, :, 3]
    img = file[:, :, 0:3]

    img, mask = crop_image(np.copy(img), np.copy(mask))

    plt.imshow(img)
    plt.show()
    plt.imshow(mask)
    plt.show()
    # print(np.unique(mask))

    # mask[mask==3] = 0
    # mask[mask==4] = 0
    # mask[mask==5] = 0
    # mask[mask==7] = 0
    # mask[mask==6] = 2

    # rgb_mask = np.zeros(shape=(mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    # rgb_mask[mask==1] = [255,0,0]
    # rgb_mask[mask==2] = [0,255,0]

    # axes[0].imshow(img[:,:,0:3])
    # axes[1].imshow(mask)
    # axes[2].imshow(rgb_mask)
    # plt.show()

In [ ]:
img_name = "TCGA-A1-A0SK-01Z-00-DX1.A44D70FA-4D96-43F4-9DD7-A61535786297.tif"
img_dir = "/home/u1910100/Documents/Tiger_Data/wsirois/wsi-level-annotations/annotations-tissue-bcss-masks"

# fig, axes = plt.subplots(1,2, figsize=(12,12))
img_path = os.path.join(img_dir, img_name)

reader = TIFFWSIReader.open(img_path)
print(reader.info.as_dict())
thumb = reader.slide_thumbnail(resolution=0, units="level")

plt.imshow(thumb)
plt.show()

In [ ]:
def extract_patches(img_name, img_dir):
    img_path = os.path.join(img_dir, img_name)
    file = np.load(img_path)

    mask = file[:, :, 3]
    img = file[:, :, 0:3]
    # img, mask = crop_image(np.copy(img), np.copy(mask))

    img_reader = VirtualWSIReader.open(img, mpp=0.5, power=20)
    mask_reader = VirtualWSIReader.open(mask, mpp=0.5, power=20)

    extractor = get_patch_extractor(
        input_img=img_reader,
        # input_mask=mask_reader,
        method_name="slidingwindow",
        patch_size=(128, 128),
        stride=(100, 100),
        resolution=20,
        units="power",
        # min_mask_ratio=0.3
    )

    results = {}
    idx = 0

    for coords in extractor.coordinate_list:
        img_patch = img_reader.read_bounds(
            coords, resolution=20, units="power"
        )
        mask_patch = mask_reader.read_bounds(
            coords, resolution=20, units="power"
        )

        img_mask_file = np.zeros(shape=(128, 128, 4), dtype=np.uint8)
        img_mask_file[:, :, 0:3] = img_patch
        img_mask_file[:, :, 3] = mask_patch[:, :, 0]

        non_zeros = np.count_nonzero(mask_patch[:, :, 0])

        img_name_without_ext = os.path.splitext(img_name)[0]

        if non_zeros > 0:
            save_name = f"{img_name_without_ext}_{idx}_p.npy"
        else:
            save_name = f"{img_name_without_ext}_{idx}_n.npy"

        results[save_name] = img_mask_file

        idx += 1

    return results

In [ ]:
fold_num = 5
tiles_dir = f"/home/u1910100/Documents/Tiger_Data/cell_detection/dilation/tiles/fold_{fold_num}"
save_dir = f"/home/u1910100/Documents/Tiger_Data/cell_detection/dilation/patches/128/fold_{fold_num}"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

img_names = os.listdir(tiles_dir)

for img_name in img_names:
    print(img_name)
    results = extract_patches(img_name, tiles_dir)
    for file_name in results.keys():
        save_path = os.path.join(save_dir, file_name)
        img_mask_file = results[file_name]

        with open(save_path, "wb") as f:
            np.save(f, img_mask_file)
print("saved")

In [ ]:
from data import TiagerDataset, TrainDataset, TestDataset, get_cell_dataloaders
from torch.utils.data import DataLoader
import torch
from torch.utils.data import random_split, ConcatDataset
import segmentation_models_pytorch as smp
import numpy as np
import os
from train import train_model
from loss_functions import BCE_Dice_Loss


patch_folder = "/media/u1910100/Extreme SSD/data/tiger/cell_detection/dilation/patches/128"
train_loader, test_loader = get_cell_dataloaders(
    patch_folder, fold_num=1, batch_size=32, phase="Train"
)

In [ ]:
from pprint import pprint

batch = next(iter(train_loader))
imgs, masks = batch["img"], batch["mask"]
# pprint(imgs.size())
# pprint(masks.size())
# pprint(file_names)

import matplotlib.pyplot as plt

for i in range(0, 32):
    img = imgs[i]
    mask = masks[i][0]

    plt.subplot(1, 2, 1)
    plt.imshow(torch.permute(img, (1, 2, 0)))
    plt.subplot(1, 2, 2)
    plt.imshow(mask, cmap="gray")
    plt.show()

# plt.imshow(torch.permute(img, (1,2,0)))
# plt.show()

# rgb_mask = np.zeros(shape=(mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
# rgb_mask[mask==1] = [255,0,0]
# rgb_mask[mask==2] = [0,255,0]

# plt.imshow(rgb_mask)
# plt.show()